In [ ]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Model
import utils.VqaQualityModel as VQM 
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
import os, json, nltk
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def encode_sentence(sentence, vocab, max_len = 14):
    unk_word = '<UNK>'
    tokens = nltk.word_tokenize(sentence.lower())
    tokens_id = [vocab.get(x, vocab[unk_word]) + 1 for x in tokens] # +1 to reserve 0 for zero paddings
    padded_tokens_id = keras.preprocessing.sequence.pad_sequences(
                        [tokens_id], maxlen=max_len, padding='post', truncating='post')

    return padded_tokens_id[0]

In [3]:
# task = vizwiz, vqa or your own task
# To create vocabulary for your own task, please refer to the sample code build_vocab.ipynb
task = 'vqa' 
vocab = json.load(open('./utils/word2vocab_{}.json'.format(task)))

In [4]:
# create the model:
# inputs
f_map = tf.placeholder(tf.float32, shape=[None, 14, 14, 2048])
q_input = tf.placeholder(tf.int32, shape=[None, 14])

# main model
model = VQM.VqaQualityModel(task)
pred = model.build_graph(f_map, q_input)
sess = tf.Session()
all_saver = tf.train.Saver() 
all_saver.restore(sess, './ckpt/vqa/cnt/ckpt') # restore from the checkpoint for the skill to be predicted

# backbone
resnet152 = keras.applications.ResNet152(include_top=False, weights='imagenet', input_shape=[448, 448, 3])
base_model = keras.models.Model(inputs=resnet152.input, outputs=resnet152.get_layer('conv5_block3_add').output)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./ckpt/vqa/cnt/-5



In [6]:
# preprocess the input image
img_dir = 'train2014'
image_name = 'COCO_train2014_000000100063.jpg'
img = image.load_img('{}/{}'.format(img_dir, image_name), target_size=(448,448)) 
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)

# preprocess the input question
question = 'Are these trains moving?'
enc_question = encode_sentence(question, vocab)

img_feat = base_model.predict(img)
output = sess.run(pred, feed_dict={f_map: img_feat, q_input: [enc_question]})

In [7]:
output

{'skills': array([[0.00016005]], dtype=float32)}